## BT Raptor: October Contest

The objective of this project is to predict the number of fatalities when dealing with Flight Disasters. The aim is to use regression to predict number of fatalities and minimize the root mean squared error.

#### WorkFlow:

1. Import Libraries
2. Load Data
3. Clean Data
4. Train the Data
5. Make Predictions and Submit 

### 1. Import Libraries

Libraries used:

- Pandas
- Numpy
- SKLearn
- Math
- Catboost
- XGBoost
- LightGBM
- Scipy

In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.linear_model import LinearRegression, Lasso, Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV, RANSACRegressor, SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, RobustScaler, MinMaxScaler, PolynomialFeatures
import xgboost as xgb
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR

from scipy.stats.mstats import winsorize

#To view the entire dataset since it's small
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### 2. Load Data

In [2]:
train = pd.read_csv('train.csv', index_col = False)
test = pd.read_csv('test.csv')

In [3]:
train.shape

(2000, 23)

In [4]:
test.shape

(423, 22)

In [5]:
# Combine Train and Test datasets in order to clean easier

data = train.append(test,sort=False)

<ipython-input-5-6a1ad5f77ca0>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = train.append(test,sort=False)


### 3. Clean Data

In [6]:
#Check how many nulls exist in the original dataset
data.isnull().sum()

Event Date                        0
Location                          6
Country                          33
Latitude                       1980
Longitude                      1980
Airport Code                   1539
Airport Name                   1491
Aircraft Damage                   0
Aircraft Category              2148
Make                              0
Amateur Built                    13
Number of Engines               342
Engine Type                     201
Schedule                       1930
Purpose of Flight               163
Air Carrier                    2256
Total Fatal Injuries            423
Total Serious Injuries            0
Total Minor Injuries              0
Total Uninjured                   0
Weather Condition                 0
Broad Phase of Flight           401
Avg_Make_Passenger_Capacity       0
dtype: int64

In [7]:
#Drop columns that contains a lot of nulls or won't be useful for our analysis

data1 = data.drop(['Latitude','Longitude','Airport Code','Airport Name','Aircraft Category','Schedule','Air Carrier',
                  'Make',
                   'Country','Location','Number of Engines','Engine Type'],axis=1)

In [13]:
# Code that created a separate column named 'States' which separated all the US states and other countries

# data1['State'] = data1['Location'].str.rsplit(',').str[-1] 
# data1['State'] = data1['State'].fillna(data1['Country'])
# data1['State'] = data1['State'].replace(['BARRANQUITAS','CULEBRA','MANATI','COMERIO','ADJUNTAS','ISBELA SEGUNDA'],'PR')
# data1['State'] = data1['State'].replace(['EUGENE ISL.BLK','EUGENE ISLAND'],'GULF OF MEXICO')
# data1['State'] = data1['State'].replace(['W. CAMERON 617'],'TX')

#Remove whitespaces from State column
# data1['State'] = data1['State'].apply(lambda x: x.strip())

In [20]:
# Replaced all nulls with UNKNOWN

data1['Purpose of Flight'] = data1['Purpose of Flight'].fillna('UNKNOWN')
data1['Broad Phase of Flight'] = data1['Broad Phase of Flight'].fillna('UNKNOWN')
data1['Amateur Built'] = data1['Amateur Built'].fillna('UNKNOWN')
# data1['Make'] = data1['Make'].fillna('UNKNOWN')
# data1['Engine Type'] = data1['Engine Type'].fillna('UNKNOWN')
# data1['Number of Engines'] = data1['Number of Engines'].fillna(data1['Number of Engines'].mode()[0])


In [21]:
#Create a year column

data1['Year'] = pd.to_datetime(data1['Event Date'])
data1['Year'] = data1['Year'].dt.year

In [22]:
#Since a year column was created, Event Date was dropped since ther is no longer have a use for it

data2 = data1.drop(['Event Date'],axis=1)

In [23]:
#Capatilize data in columns to keep letters consistent
Upper_cols = ['Aircraft Damage',
       'Amateur Built',  
       'Purpose of Flight', 
        'Weather Condition','Broad Phase of Flight']
#        'Make',]

for cols in Upper_cols:
    data2[cols] = data2[cols].str.upper()

In [41]:
# Create dummy variables and drop the first column to avoid Dummy Variable Trap
data3 = pd.get_dummies(data2,drop_first = True)

### 5. Train the Data

Spit the data back into a train and test split and removed the highest and lowest outliers

Trained using normal and scaled data on several regression model to see which ones performs the best.

In [43]:
#Split back data into new Train and Test Split
newTrain = data3.iloc[:2000]
newTest = data3.iloc[2000:]

In [44]:
# Created a new column 'avg_survivor' that calculated a ratio of all survivors against the average model capacity

newTrain['avg_survivor'] = (newTrain['Total Serious Injuries'] + newTrain['Total Minor Injuries'] + newTrain['Total Uninjured'])/  newTrain['Avg_Make_Passenger_Capacity']
newTest['avg_survivor'] = (newTest['Total Serious Injuries'] + newTest['Total Minor Injuries'] + newTest['Total Uninjured'])/  newTest['Avg_Make_Passenger_Capacity']

<ipython-input-44-e312acdd10d7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newTrain['avg_surviver'] = (newTrain['Total Serious Injuries'] + newTrain['Total Minor Injuries'] + newTrain['Total Uninjured'])/  newTrain['Avg_Make_Passenger_Capacity']
<ipython-input-44-e312acdd10d7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newTest['avg_surviver'] = (newTest['Total Serious Injuries'] + newTest['Total Minor Injuries'] + newTest['Total Uninjured'])/  newTest['Avg_Make_Passenger_Capacity']


In [45]:
newTest = newTest.drop('Total Fatal Injuries',axis=1)

In [48]:
#Split the training data in order to do some tests

X = newTrain.drop(['Total Fatal Injuries'], axis=1)
y = newTrain['Total Fatal Injuries']

In [49]:
# Remove the top 5% and bottom 5% of outliers for our dependent variable 
y = winsorize(y,(0.05,0.05))

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=0)

In [53]:
# Also testing using normalized data

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_scaled = scaler.fit_transform(X)
newTest_scaled = scaler.transform(newTest)

In [54]:
#Functions to do model predictions for normal and scaled data

def model_predicitons(model_to_use,X_train,y_train,X_test,y_test):
    model = model_to_use
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    print(model)
    print('MSE: ', mean_squared_error(y_test, pred))
    print('RMSE: ', sqrt(mean_squared_error(y_test, pred)))
    
def model_predicitons_scaled(model,X_train_scaled,y_train,X_test_scaled,y_test):
    model.fit(X_train_scaled,y_train)
    pred = model.predict(X_test_scaled)
    print(model)
    print('MSE scaled: ', mean_squared_error(y_test, pred))
    print('RMSE scaled: ', sqrt(mean_squared_error(y_test, pred)))

In [55]:

models = [LinearRegression(),
          xgb.XGBRegressor(),
         CatBoostRegressor(),
         LGBMRegressor(),
         Lasso(),
         Ridge(),
        # KNeighborsRegressor(),
         RandomForestRegressor(),
         DecisionTreeRegressor(),
         RidgeCV(), 
         ElasticNet(), 
         LassoCV(), 
         LassoLarsCV(),
         RANSACRegressor(),
         #PolynomialFeatures(),
         SGDRegressor(),
         SVR()]
          

In [56]:
for model in models:
    model_predicitons(model,X_train, y_train, X_test,  y_test)
    print('')

LinearRegression()
MSE:  3.6574794705632665
RMSE:  1.9124537825953511

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
MSE:  6.648908381334638
RMSE:  2.578547727177963

Learning rate set to 0.044099


C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0:	learn: 3.0968548	total: 143ms	remaining: 2m 22s
1:	learn: 3.0440916	total: 144ms	remaining: 1m 11s
2:	learn: 2.9917622	total: 145ms	remaining: 48s
3:	learn: 2.9401307	total: 145ms	remaining: 36.1s
4:	learn: 2.8918451	total: 146ms	remaining: 29s
5:	learn: 2.8481961	total: 146ms	remaining: 24.2s
6:	learn: 2.8077002	total: 147ms	remaining: 20.8s
7:	learn: 2.7666727	total: 147ms	remaining: 18.3s
8:	learn: 2.7317916	total: 148ms	remaining: 16.3s
9:	learn: 2.7008635	total: 148ms	remaining: 14.7s
10:	learn: 2.6692294	total: 149ms	remaining: 13.4s
11:	learn: 2.6378063	total: 149ms	remaining: 12.3s
12:	learn: 2.6102562	total: 150ms	remaining: 11.4s
13:	learn: 2.5798161	total: 150ms	remaining: 10.6s
14:	learn: 2.5557313	total: 151ms	remaining: 9.91s
15:	learn: 2.5295524	total: 151ms	remaining: 9.32s
16:	learn: 2.5064821	total: 152ms	remaining: 8.79s
17:	learn: 2.4820045	total: 153ms	remaining: 8.33s
18:	learn: 2.4601586	total: 153ms	remaining: 7.91s
19:	learn: 2.4397590	total: 154ms	remaining

282:	learn: 1.8971824	total: 305ms	remaining: 772ms
283:	learn: 1.8962746	total: 306ms	remaining: 770ms
284:	learn: 1.8953497	total: 306ms	remaining: 768ms
285:	learn: 1.8952798	total: 307ms	remaining: 766ms
286:	learn: 1.8944205	total: 307ms	remaining: 764ms
287:	learn: 1.8937347	total: 308ms	remaining: 762ms
288:	learn: 1.8931351	total: 309ms	remaining: 760ms
289:	learn: 1.8927261	total: 309ms	remaining: 758ms
290:	learn: 1.8918114	total: 310ms	remaining: 755ms
291:	learn: 1.8904625	total: 311ms	remaining: 753ms
292:	learn: 1.8900467	total: 311ms	remaining: 751ms
293:	learn: 1.8896580	total: 312ms	remaining: 749ms
294:	learn: 1.8895908	total: 312ms	remaining: 746ms
295:	learn: 1.8891499	total: 313ms	remaining: 744ms
296:	learn: 1.8879079	total: 314ms	remaining: 742ms
297:	learn: 1.8877419	total: 314ms	remaining: 740ms
298:	learn: 1.8875041	total: 315ms	remaining: 738ms
299:	learn: 1.8869720	total: 315ms	remaining: 736ms
300:	learn: 1.8858337	total: 316ms	remaining: 734ms
301:	learn: 

564:	learn: 1.7701454	total: 472ms	remaining: 364ms
565:	learn: 1.7699372	total: 473ms	remaining: 363ms
566:	learn: 1.7697488	total: 474ms	remaining: 362ms
567:	learn: 1.7693489	total: 474ms	remaining: 361ms
568:	learn: 1.7690155	total: 475ms	remaining: 360ms
569:	learn: 1.7683825	total: 476ms	remaining: 359ms
570:	learn: 1.7679999	total: 476ms	remaining: 358ms
571:	learn: 1.7676781	total: 477ms	remaining: 357ms
572:	learn: 1.7670866	total: 477ms	remaining: 356ms
573:	learn: 1.7668123	total: 478ms	remaining: 355ms
574:	learn: 1.7664568	total: 479ms	remaining: 354ms
575:	learn: 1.7660593	total: 479ms	remaining: 353ms
576:	learn: 1.7657172	total: 480ms	remaining: 352ms
577:	learn: 1.7653963	total: 481ms	remaining: 351ms
578:	learn: 1.7650665	total: 481ms	remaining: 350ms
579:	learn: 1.7648262	total: 482ms	remaining: 349ms
580:	learn: 1.7645408	total: 483ms	remaining: 348ms
581:	learn: 1.7643481	total: 483ms	remaining: 347ms
582:	learn: 1.7641483	total: 484ms	remaining: 346ms
583:	learn: 

846:	learn: 1.7085581	total: 640ms	remaining: 116ms
847:	learn: 1.7083018	total: 641ms	remaining: 115ms
848:	learn: 1.7082091	total: 642ms	remaining: 114ms
849:	learn: 1.7080726	total: 642ms	remaining: 113ms
850:	learn: 1.7080573	total: 643ms	remaining: 113ms
851:	learn: 1.7080502	total: 643ms	remaining: 112ms
852:	learn: 1.7079691	total: 644ms	remaining: 111ms
853:	learn: 1.7076552	total: 645ms	remaining: 110ms
854:	learn: 1.7073940	total: 645ms	remaining: 109ms
855:	learn: 1.7073534	total: 646ms	remaining: 109ms
856:	learn: 1.7071911	total: 647ms	remaining: 108ms
857:	learn: 1.7071191	total: 647ms	remaining: 107ms
858:	learn: 1.7070081	total: 648ms	remaining: 106ms
859:	learn: 1.7069965	total: 648ms	remaining: 106ms
860:	learn: 1.7069598	total: 649ms	remaining: 105ms
861:	learn: 1.7069205	total: 650ms	remaining: 104ms
862:	learn: 1.7067684	total: 650ms	remaining: 103ms
863:	learn: 1.7065882	total: 651ms	remaining: 102ms
864:	learn: 1.7064605	total: 651ms	remaining: 102ms
865:	learn: 

MSE:  4.311270801178058
RMSE:  2.0763599883397044

Lasso(alpha=0.0291505)
MSE:  3.661304443883131
RMSE:  1.9134535384699392

Ridge()
MSE:  3.6338159896131197
RMSE:  1.9062570628362587

RandomForestRegressor()
MSE:  5.196134753329827
RMSE:  2.27950318125021

DecisionTreeRegressor()
MSE:  7.482685329202567
RMSE:  2.73544974898143

RidgeCV()
MSE:  3.583130413236022
RMSE:  1.8929158494861893

ElasticNet()
MSE:  3.8308143196553397
RMSE:  1.9572466169737885

LassoCV()
MSE:  3.6584755361326633
RMSE:  1.9127141804599723

LassoLarsCV()
MSE:  3.5303210954909092
RMSE:  1.878914871805242

RANSACRegressor()
MSE:  34.38911758780484
RMSE:  5.864223528124149

SGDRegressor()
MSE:  4.0120227505603624e+24
RMSE:  2003003432488.4126



C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsCV())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


SVR()
MSE:  3.9641873692497858
RMSE:  1.9910267123395873



In [57]:
for model in models:
    model_predicitons_scaled(model,X_train_scaled, y_train, X_test_scaled,  y_test)
    print('')

LinearRegression()
MSE scaled:  3.6574794705632656
RMSE scaled:  1.912453782595351

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=12,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
MSE scaled:  6.648908381334638
RMSE scaled:  2.578547727177963

Learning rate set to 0.044099
0:	learn: 3.0968548	total: 577us	remaining: 577ms
1:	learn: 3.0440916	total: 1.25ms	remaining: 625ms
2:	learn: 2.9917622	total: 1.91ms	remaining: 634ms
3:	learn: 2.9401307	total: 2.56ms	remain

342:	learn: 1.8641462	total: 214ms	remaining: 410ms
343:	learn: 1.8632278	total: 215ms	remaining: 410ms
344:	learn: 1.8627161	total: 216ms	remaining: 409ms
345:	learn: 1.8621821	total: 216ms	remaining: 409ms
346:	learn: 1.8616369	total: 217ms	remaining: 408ms
347:	learn: 1.8609485	total: 218ms	remaining: 408ms
348:	learn: 1.8606342	total: 218ms	remaining: 407ms
349:	learn: 1.8599470	total: 219ms	remaining: 407ms
350:	learn: 1.8597375	total: 220ms	remaining: 406ms
351:	learn: 1.8593571	total: 220ms	remaining: 405ms
352:	learn: 1.8590222	total: 221ms	remaining: 405ms
353:	learn: 1.8586551	total: 222ms	remaining: 404ms
354:	learn: 1.8582767	total: 222ms	remaining: 404ms
355:	learn: 1.8581920	total: 223ms	remaining: 403ms
356:	learn: 1.8577467	total: 223ms	remaining: 402ms
357:	learn: 1.8574715	total: 224ms	remaining: 402ms
358:	learn: 1.8569470	total: 225ms	remaining: 401ms
359:	learn: 1.8565896	total: 226ms	remaining: 401ms
360:	learn: 1.8559357	total: 226ms	remaining: 401ms
361:	learn: 

609:	learn: 1.7551011	total: 382ms	remaining: 244ms
610:	learn: 1.7550225	total: 382ms	remaining: 243ms
611:	learn: 1.7548329	total: 383ms	remaining: 243ms
612:	learn: 1.7544529	total: 384ms	remaining: 242ms
613:	learn: 1.7539549	total: 384ms	remaining: 242ms
614:	learn: 1.7536990	total: 385ms	remaining: 241ms
615:	learn: 1.7533519	total: 386ms	remaining: 240ms
616:	learn: 1.7531318	total: 386ms	remaining: 240ms
617:	learn: 1.7529755	total: 387ms	remaining: 239ms
618:	learn: 1.7528331	total: 387ms	remaining: 238ms
619:	learn: 1.7524914	total: 388ms	remaining: 238ms
620:	learn: 1.7523622	total: 389ms	remaining: 237ms
621:	learn: 1.7522741	total: 389ms	remaining: 237ms
622:	learn: 1.7520585	total: 390ms	remaining: 236ms
623:	learn: 1.7518925	total: 390ms	remaining: 235ms
624:	learn: 1.7517173	total: 391ms	remaining: 235ms
625:	learn: 1.7512802	total: 392ms	remaining: 234ms
626:	learn: 1.7508660	total: 392ms	remaining: 233ms
627:	learn: 1.7507864	total: 393ms	remaining: 233ms
628:	learn: 

881:	learn: 1.7044874	total: 550ms	remaining: 73.5ms
882:	learn: 1.7044330	total: 550ms	remaining: 72.9ms
883:	learn: 1.7043337	total: 551ms	remaining: 72.3ms
884:	learn: 1.7042782	total: 552ms	remaining: 71.7ms
885:	learn: 1.7041317	total: 552ms	remaining: 71.1ms
886:	learn: 1.7040813	total: 553ms	remaining: 70.4ms
887:	learn: 1.7039556	total: 553ms	remaining: 69.8ms
888:	learn: 1.7037586	total: 554ms	remaining: 69.2ms
889:	learn: 1.7036951	total: 555ms	remaining: 68.6ms
890:	learn: 1.7036315	total: 555ms	remaining: 67.9ms
891:	learn: 1.7031678	total: 556ms	remaining: 67.3ms
892:	learn: 1.7031108	total: 557ms	remaining: 66.7ms
893:	learn: 1.7029703	total: 557ms	remaining: 66.1ms
894:	learn: 1.7029642	total: 558ms	remaining: 65.4ms
895:	learn: 1.7029209	total: 558ms	remaining: 64.8ms
896:	learn: 1.7023578	total: 559ms	remaining: 64.2ms
897:	learn: 1.7022402	total: 560ms	remaining: 63.6ms
898:	learn: 1.7021631	total: 560ms	remaining: 62.9ms
899:	learn: 1.7019249	total: 561ms	remaining: 

C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RidgeCV was fitted with feature names
  warnings.warn(
C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:133: FutureWarning: The default of 'normalize' will be set to False in version 1.2 and deprecated in version 1.4.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LassoLarsCV())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


SVR()
MSE scaled:  3.624645496966331
RMSE scaled:  1.9038501771322056



#### Tuning Alpha for the Lasso Regression

Since Lasso Regression performed well, tried to tune it to improve performance. Tried tuning using various values of alpha.

In [64]:
params = {'alpha': (np.logspace(-8, 8, 100))}
lasso = Lasso(normalize=True)
lasso_model = GridSearchCV(lasso, params, cv = 10)
lasso_model.fit(X_train, y_train)
print(lasso_model.best_params_)
print(lasso_model.best_score_)

C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.287e+03, tolerance: 1.463e+00
  model = cd_fast.enet_coordinate_descent(
C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X

...

C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To

C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To

C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To

{'alpha': 0.0014849682622544665}
0.4453291136919663


C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
C:\Users\navin\anaconda3\envs\Data Science\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To

### 5. Make Predictions and Submit 

After training and tuning now make our final training using X_scaled and y. Then predict using newTestscaled.

The submit_to_kaggle function is used to create a .csv that we then submit on Kaggle.

In [65]:
def submit_to_kaggle(name,pred):
    submission = pd.read_csv('sample_submission.csv')
    submission['Total Fatal Injuries'] = pred
    submission.to_csv(name + '.csv',index=None)

In [66]:
# Train Model using entire dataset then predict
model = Lasso(alpha = 0.0291505)
model.fit(X_scaled,y)
pred = model.predict(newTest_scaled)

In [67]:
#Create Kaggle output
submit_to_kaggle('lasso_v2',pred)